# Neural Arithmetic Logic Unit

This notebook contains basic experiments with the NAC and NALU activation units. We attempt to reproduce some of the results found in [arXiv:1808.00508](https://arxiv.org/abs/1808.00508).

In [1]:
import keras as k
from keras.models import *
import keras.backend as K

C:\Users\acyrl\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import nalu.GenData as gd

In [3]:
from nalu.NALU import NALU
from nalu.NAC import NAC

In [4]:
def pred_close(y_true, y_pred):
    return K.abs(y_true - y_pred) < 1e-4

## Idendity

### NAC


In [43]:
_, Y = gd.gd_paper(size=100000)

In [46]:
x = Input((1,)) # n is the dimension of the data, (n, N)
y = NAC(1)(x) # number of units, but let's test
m = Model(x, y)
m.compile(k.optimizers.RMSprop(lr=0.01), "mse", metrics=[pred_close])
m.fit(Y, Y, batch_size=20, epochs=10)

Epoch 1/10
100000/100000 [==============================] - 7s 67us/step - loss: 919.9096 - pred_close: 0.5885
Epoch 2/10
100000/100000 [==============================] - 6s 64us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 3/10
100000/100000 [==============================] - 6s 64us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 4/10
100000/100000 [==============================] - 6s 63us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 5/10
100000/100000 [==============================] - 6s 63us/step - loss: 2.1185e-11 - pred_close: 1.0000
Epoch 6/10
100000/100000 [==============================] - 6s 64us/step - loss: 0.0000e+00 - pred_close: 1.0000
Epoch 7/10
100000/100000 [==============================] - 6s 63us/step - loss: 0.0000e+00 - pred_close: 1.0000
Epoch 8/10
100000/100000 [==============================] - 6s 64us/step - loss: 0.0000e+00 - pred_close: 1.0000
Epoch 9/10
100000/100000 [==============================] - 6s 64us/step - loss: 0.0000e+00 - pred

In [54]:
_, Yext = gd.gd_uniform(500000,550000, size=50000)
m.evaluate(Yext, Yext)

50000/50000 [==============================] - 1s 17us/step


[0.0, 1.0]

In [55]:
x = Input((1,)) # n is the dimension of the data, (n, N)
y = NALU(1)(x) # number of units, but let's test
m = Model(x, y)
m.compile(k.optimizers.RMSprop(lr=0.01), "mse", metrics=[pred_close])
m.fit(Y, Y, batch_size=20, epochs=10)

Epoch 1/10
100000/100000 [==============================] - 9s 86us/step - loss: 2240.6077 - pred_close: 0.5737
Epoch 2/10
100000/100000 [==============================] - 8s 84us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 3/10
100000/100000 [==============================] - 8s 82us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 4/10
100000/100000 [==============================] - 8s 83us/step - loss: 3.5726e-10 - pred_close: 1.0000
Epoch 5/10
100000/100000 [==============================] - 8s 84us/step - loss: 2.6859e-11 - pred_close: 1.0000
Epoch 6/10
100000/100000 [==============================] - 8s 83us/step - loss: 5.7103e-18 - pred_close: 1.0000
Epoch 7/10
100000/100000 [==============================] - 8s 84us/step - loss: 5.7103e-18 - pred_close: 1.0000
Epoch 8/10
100000/100000 [==============================] - 8s 84us/step - loss: 5.7103e-18 - pred_close: 1.0000
Epoch 9/10
100000/100000 [==============================] - 9s 86us/step - loss: 5.7103e-18 - pre

In [57]:
_, Yext = gd.gd_uniform(500000,550000, size=50000)
m.evaluate(Yext, Yext)


50000/50000 [==============================] - 1s 21us/step


[<tf.Variable 'nalu_7/W_hat:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'nalu_7/M_hat:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'nalu_7/G:0' shape=(1, 1) dtype=float32_ref>]

In [64]:
m.get_weights()

[array([[9.012543]], dtype=float32),
 array([[16.635576]], dtype=float32),
 array([[0.8285337]], dtype=float32)]

## Addition

### NALU/paper

In [33]:
X, Y = gd.gd_paper(size=100000)

In [36]:
x = Input((2,)) # n is the dimension of the data, (n, N)
y = NALU(1)(x) # number of units, but let's test
m = Model(x, y)
m.compile(k.optimizers.RMSprop(lr=0.01), "mse", metrics=[pred_close])
m.fit(X, Y, batch_size=20, epochs=50)

Epoch 1/50
100000/100000 [==============================] - 8s 85us/step - loss: 1052.5409 - pred_close: 0.5276
Epoch 2/50
100000/100000 [==============================] - 8s 83us/step - loss: 5.9485e-07 - pred_close: 0.9992
Epoch 3/50
100000/100000 [==============================] - 9s 88us/step - loss: 3.0850e-07 - pred_close: 0.9995
Epoch 4/50
100000/100000 [==============================] - 8s 83us/step - loss: 2.0361e-07 - pred_close: 0.9996
Epoch 5/50
100000/100000 [==============================] - 9s 85us/step - loss: 1.5973e-07 - pred_close: 0.9997
Epoch 6/50
100000/100000 [==============================] - 9s 85us/step - loss: 1.3526e-07 - pred_close: 0.9997
Epoch 7/50
100000/100000 [==============================] - 8s 81us/step - loss: 1.1671e-07 - pred_close: 0.9997
Epoch 8/50
100000/100000 [==============================] - 8s 84us/step - loss: 1.0152e-07 - pred_close: 0.9997
Epoch 9/50
100000/100000 [==============================] - 8s 83us/step - loss: 9.0443e-08 - pre

#### Interpolation

It looks like there could be some overfitting to the training set. Do we need to train on uniform?

In [37]:
Xint, Yint = gd.gd_paper(size=50000)
m.evaluate(Xint, Yint)

50000/50000 [==============================] - 1s 20us/step


[4.6140787588626605e-08, 0.99982]

In [38]:
Xint, Yint = gd.gd_uniform(size=50000)
m.evaluate(Xint, Yint)

50000/50000 [==============================] - 1s 19us/step


[2.1089257575710006e-06, 0.97478]

#### Extrapolation

In [39]:
Xext, Yext = gd.gd_uniform(size=50000)
m.evaluate(Xext, Yext)

50000/50000 [==============================] - 1s 19us/step


[2.13256411180609e-06, 0.97538]

In [41]:
Xext, Yext = gd.gd_uniform(30000, 35000, size=50000)
m.evaluate(Xext, Yext)

50000/50000 [==============================] - 1s 20us/step


[7.498779296875e-06, 0.75252]